In [2]:
import numpy as np
import pickle
from tqdm import tqdm
import os
import pypianoroll as pr
from Music_Style_Transfer_master.project.utils import dataset_import, padding_v2, get_metas

In [66]:
beat_resolution = 8
len_context = beat_resolution*4*8
max_len = beat_resolution*4*4
# 33:97 A1~C7,  21:109 A0~C8, 
# 7 octave 21:105  A0~G#7  
pitch_range = np.arange(21, 105)
scaler = np.arange(1, 85)

merge = [0 ,1, 2, 3]   # bach: [0 ,1, 2, 3], LPD5: [1, 2, 3 ,4], Medium_swing: [0, 1]
path = "Bach_dataset/JSB_Chorales_music21/" 
output_path =  "../../../scratch/wtl272/dataset/bach_8_on_offset/"   #"../../../scratch/wtl272/dataset/lpd_5/lpd_5_cleansed_score/"
file_list = dataset_import(path, ".mid")

# Parse to pianoroll
file_index = 0
for i, ii in tqdm(enumerate(file_list)):
    tmp = pr.Multitrack(filename=ii, beat_resolution=beat_resolution)
    # Create a merge track without drums
    tmp.merge_tracks(merge)
    tmp.binarize()
    tmp.assign_constant(1)

    score = tmp.tracks[-1].pianoroll[:, pitch_range]
    
    # Exception
    if not score.shape[0]:
        continue
        
    pitch_table = np.array(score[0])
    pitch_counter = np.zeros(len(pitch_range), dtype="int64")
    score_progress = (np.concatenate([score[1:], np.zeros((1, len(pitch_range)))]) - score).astype('int64')
    # Intensity as length
    # 1 as onset, gradually decrease to zero 
    new_score = np.zeros((score.shape[0], score.shape[1], 3))
    for t ,tt in enumerate(score):
        for p in np.nonzero(pitch_table)[0]:
            if pitch_table[p] and pitch_counter[p] == 0:
                new_score[t, p, 0] = 1
                pitch_counter[p] = 1   
            
            elif pitch_table[p] == -1:
                new_score[t, p, 1] = 1
                pitch_counter[p] -= 1
                new_score[t - pitch_counter[p]:t, p, 2] = new_score[t - pitch_counter[p]:t, p, 2][::-1]
                pitch_counter[p] = 0
            
        for p in np.nonzero(pitch_counter)[0]:
            new_score[t, p, 2] = pitch_counter[p]
            pitch_counter[p] += 1
            
        pitch_table = score_progress[t]
        
    t +=1
    for p in np.nonzero(pitch_table)[0]:
        if pitch_table[p] == -1:
            pitch_counter[p] -= 1
            new_score[t - pitch_counter[p]:t, p, 2] = new_score[t - pitch_counter[p]:t, p, 2][::-1]

    iii = 0
    score_t = np.roll(new_score, shift=iii, axis=1)
    score_t  = padding_v2(score_t , len_context)
    np.save(output_path +str(file_index) + "_" + str(iii) + ".npy", score_t)
        
    timegrid  = get_metas(score_t, beat_resolution, len_context)
    np.save(output_path + str(file_index) + "_meta.npy", timegrid)
    
    file_index += 1

0it [00:00, ?it/s]

Load from Bach_dataset/JSB_Chorales_music21/
371 files loaded!


/home/wtl272/py3/lib/python3.6/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)
371it [00:40,  9.11it/s]
